In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
news_df=pd.read_csv('train.csv')

In [4]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


news_df.shape()

In [5]:
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
news_df=news_df.fillna(' ')

In [7]:
news_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
news_df['content']=news_df['author']+" "+news_df['title']

In [9]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [10]:
news_df['content'][20796]

'Benjamin Hoffman N.F.L. Playoffs: Schedule, Matchups and Odds - The New York Times'

In [11]:
ps=PorterStemmer()

def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]'," ",content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=" ".join(stemmed_content)
    return stemmed_content

In [12]:
news_df['content'] = news_df['content'].apply(stemming)

In [14]:
news_df['content'].values

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [15]:
X=news_df['content'].values
Y=news_df['label'].values

In [16]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [17]:
vector=TfidfVectorizer()
vector.fit(X)
X=vector.transform(X)

In [18]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210687 stored elements and shape (20800, 17128)>
  Coords	Values
  (0, 267)	0.2701012497770876
  (0, 2483)	0.36765196867972083
  (0, 2959)	0.24684501285337127
  (0, 3600)	0.3598939188262558
  (0, 3792)	0.27053324808454915
  (0, 4973)	0.23331696690935097
  (0, 7005)	0.2187416908935914
  (0, 7692)	0.24785219520671598
  (0, 8630)	0.2921251408704368
  (0, 8909)	0.36359638063260746
  (0, 13473)	0.2565896679337956
  (0, 15686)	0.2848506356272864
  (1, 1497)	0.2939891562094648
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 2813)	0.19094574062359204
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 16799)	0.30071745655510157
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 3643)	0.2115550061362374
  (20797, 7042)	0.21799048897828685
  (2079

In [19]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=1)

In [20]:
X_train.shape

(16640, 17128)

In [21]:
X_test.shape

(4160, 17128)

In [22]:
model=LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [23]:
train_y_pred=model.predict(X_train)
print("train accuracy : ",accuracy_score(train_y_pred,Y_train))

train accuracy :  0.9868389423076923


In [24]:
test_y_pred=model.predict(X_test)
print("train accuracy : ",accuracy_score(test_y_pred,Y_test))

train accuracy :  0.9764423076923077


In [25]:
print(classification_report(Y_test,test_y_pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      2077
           1       0.96      0.99      0.98      2083

    accuracy                           0.98      4160
   macro avg       0.98      0.98      0.98      4160
weighted avg       0.98      0.98      0.98      4160



In [26]:
DTC=DecisionTreeClassifier()

In [27]:
DTC.fit(X_train,Y_train)

DecisionTreeClassifier()

In [28]:
pred_dtc=DTC.predict(X_test)

In [29]:
DTC.score(X_test,Y_test)

0.9913461538461539

In [30]:
print(classification_report(Y_test,pred_dtc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2077
           1       0.99      0.99      0.99      2083

    accuracy                           0.99      4160
   macro avg       0.99      0.99      0.99      4160
weighted avg       0.99      0.99      0.99      4160



In [31]:
rfc=RandomForestClassifier()

In [32]:
rfc.fit(X_train, Y_train)


RandomForestClassifier()

In [33]:
rfc.score(X_test,Y_test)

0.9942307692307693

In [34]:
pred_rfc=rfc.predict(X_test)

In [35]:
print(classification_report(Y_test,pred_rfc))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      2077
           1       0.99      1.00      0.99      2083

    accuracy                           0.99      4160
   macro avg       0.99      0.99      0.99      4160
weighted avg       0.99      0.99      0.99      4160



In [36]:
import pickle

In [37]:
with open('fake_news_model.pkl','wb') as model_file:
    pickle.dump(model,model_file)

In [38]:
with open('tfidf_vectorizer.pkl','wb') as vec_file:
    pickle.dump(vector,vec_file)

In [39]:
with open('fake_news_model.pkl','rb') as model_file:
    loaded_model=pickle.load(model_file)

In [40]:
with open('tfidf_vectorizer.pkl','rb') as vec_file:
    loaded_vector=pickle.load(vec_file)

In [41]:
X_new_text=["This might be fake news"]
X_new=loaded_vector.transform(X_new_text)
predictions=loaded_model.predict(X_new)
print("Predictions:",predictions)

Predictions: [1]


In [50]:
input_data = X_test[1560]
prediction=model.predict(input_data)
if prediction[0]==1:
    print("Fake News")
else:
    print("Real News")

Real News


In [48]:
news_df['content'][1560]

'manohla dargi director toni erdmann savor moment cann new york time'